In [3]:
import sqlite3
db = sqlite3.connect("./data_gatherer/fintime.db",check_same_thread=False)

In [4]:
cursor = db.cursor()
cursor.execute("SELECT ticker,dataJSON FROM stockData")
stockData = cursor.fetchall()

In [6]:
import json
data = []
for row in stockData:
    dataaa = json.loads(row[1])
    dataaa['ticker'] = row[0]
    data.append(dataaa)

In [7]:
from datetime import datetime
def getDataForDaysHistorical(hist, daysStart, daysEnd):
    midPoint = int(len(hist) / 2)
    startIndex = midPoint + daysStart
    endIndex = midPoint + daysEnd
    data = hist[startIndex:endIndex]
    return data, hist[midPoint]

def getDaysFrumCurrentD(hist):
    midPoint = int(len(hist) / 2)

    return hist[midPoint]["Date"]

def getDaysFrumCurrent(hist):
        date_str = getDaysFrumCurrentD(hist)
        date_obj = datetime.strptime(date_str, "%Y-%m-%d")
        current_date = datetime.now()
        difference = (current_date - date_obj).days
        return difference


def getLowestLow(hist):
    lowest = hist[0]["Low"]
    for day in hist:
        if day["Low"] < lowest:
            lowest = day["Low"]
    return lowest
def getDividends(hist):
    dividends = 0
    for day in hist:
        if day["Dividends"] != 0:
            dividends += day["Dividends"]
    return dividends
def getHighestHigh(hist):
    highest = hist[0]["High"]
    for day in hist:
        if day["High"] > highest:
            highest = day["High"]
    return highest


def getLowAndHighForDays(hist, start, end):
    newHist,midpoint = getDataForDaysHistorical(hist, start, end)
    return getLowestLow(newHist), getHighestHigh(newHist),getDividends(newHist),midpoint["Date"]

def getDataForPercentageRange(hist, startPercent, endPercent):
    midPoint = len(hist) // 2
    startIndex = midPoint + int(midPoint * startPercent / 100)
    endIndex = midPoint + int(midPoint * endPercent / 100)
    startIndex = max(0, startIndex)
    endIndex = min(len(hist), endIndex)
    data = hist[startIndex:endIndex]
    return getHighestHigh(data), getLowestLow(data), getDividends(data)

print(getDaysFrumCurrent(data[0]["hist"]))

87


In [8]:
getDataForPercentageRange(data[0]["hist"],-50,-20)

(237.76597136065524, 202.7194522842973, 0)

In [ ]:
err=0
for dataP in data:
        try:
            getDataForPercentageRange(dataP["hist"], -100, -80)[0]
            getDataForPercentageRange(dataP["hist"], -80, -60)[1]
            getDataForPercentageRange(dataP["hist"], -60, -40)[0]
            getDataForPercentageRange(dataP["hist"], -60, -40)[1]
            getDataForPercentageRange(dataP["hist"], -40, -20)[0]
            getDataForPercentageRange(dataP["hist"], -40, -20)[1]
            getDataForPercentageRange(dataP["hist"], -20, -0)[0]
            getDataForPercentageRange(dataP["hist"], -20, -0)[1]

            getDataForPercentageRange(dataP["hist"], 0, 20)
            getDataForPercentageRange(dataP["hist"], 0, 100)
        except:
              err+=1
              data.remove(dataP)
err

In [ ]:
newData=[]
results = []
for company in data:
        result = []
        extra = company.get("extra", {})
        
        # Extract numeric features from 'extra' section
        fiscal_year = extra.get("Fiscal Year", {})
        profitability = extra.get("Profitability", {})
        management = extra.get("Management Effectiveness", {})
        income_statement = extra.get("Income Statement", {})
        balance_sheet = extra.get("Balance Sheet", {})
        cash_flow = extra.get("Cash Flow Statement", {})
        stock_history = extra.get("Stock Price History", {})
        share_stats = extra.get("Share Statistics", {})
        dividends_splits = extra.get("Dividends & Splits", {})
        
        result = [
            company.get("current_price", 0) or -1,
            company.get("Market Cap", 0) or -1,
            company.get("Enterprise Value", 0) or -1,
            company.get("Trailing P/E", 0) or -1,
            company.get("Forward P/E", 0) or -1,
            company.get("PEG Ratio (5yr expected)", 0) or -1,
            company.get("Price/Sales", 0) or -1,
            company.get("Price/Book", 0) or -1,
            company.get("Enterprise Value/Revenue", 0) or -1,
            company.get("Enterprise Value/EBITDA", 0) or -1,
            getDataForPercentageRange(company.get("hist"), -100, -80)[0],
            getDataForPercentageRange(company.get("hist"), -80, -60)[1],
            getDataForPercentageRange(company.get("hist"), -60, -40)[0],
            getDataForPercentageRange(company.get("hist"), -60, -40)[1],
            getDataForPercentageRange(company.get("hist"), -40, -20)[0],
            getDataForPercentageRange(company.get("hist"), -40, -20)[1],
            getDataForPercentageRange(company.get("hist"), -20, -0)[0],
            getDataForPercentageRange(company.get("hist"), -20, -0)[1],

            float(profitability.get("Profit Margin", -1)) if profitability.get("Profit Margin") is not None else -1.0,
            float(profitability.get("Operating Margin  (ttm)", -1)) if profitability.get("Operating Margin  (ttm)") is not None else -1.0,
            # Management Effectiveness
            float(management.get("Return on Assets  (ttm)", -1)) if management.get("Return on Assets  (ttm)") is not None else -1.0,
            float(management.get("Return on Equity  (ttm)", -1)) if management.get("Return on Equity  (ttm)") is not None else -1.0,
            # Income Statement
            float(income_statement.get("Revenue  (ttm)", 0)) if income_statement.get("Revenue  (ttm)") is not None else -1.0,
            float(income_statement.get("Revenue Per Share  (ttm)", 0)) if income_statement.get("Revenue Per Share  (ttm)") is not None else -1.0,
            float(income_statement.get("Quarterly Revenue Growth  (yoy)", 0)) if income_statement.get("Quarterly Revenue Growth  (yoy)") is not None else -1.0,
            float(income_statement.get("Gross Profit  (ttm)", 0)) if income_statement.get("Gross Profit  (ttm)") is not None else -1.0,
            float(income_statement.get("EBITDA", 0)) if income_statement.get("EBITDA") is not None else -1.0,
            float(income_statement.get("Net Income Avi to Common  (ttm)", 0)) if income_statement.get("Net Income Avi to Common  (ttm)") is not None else -1.0,
            float(income_statement.get("Diluted EPS  (ttm)", 0)) if income_statement.get("Diluted EPS  (ttm)") is not None else -1.0,
            float(income_statement.get("Quarterly Earnings Growth  (yoy)", 0)) if income_statement.get("Quarterly Earnings Growth  (yoy)") is not None else -1.0,
            # Balance Sheet
            float(balance_sheet.get("Total Cash  (mrq)", 0)) if balance_sheet.get("Total Cash  (mrq)") is not None else -1.0,
            float(balance_sheet.get("Total Cash Per Share  (mrq)", 0)) if balance_sheet.get("Total Cash Per Share  (mrq)") is not None else -1.0,
            float(balance_sheet.get("Total Debt  (mrq)", 0)) if balance_sheet.get("Total Debt  (mrq)") is not None else -1.0,
            float(balance_sheet.get("Total Debt/Equity  (mrq)", 0)) if balance_sheet.get("Total Debt/Equity  (mrq)") is not None else -1.0,
            float(balance_sheet.get("Current Ratio  (mrq)", 0)) if balance_sheet.get("Current Ratio  (mrq)") is not None else -1.0,
            float(balance_sheet.get("Book Value Per Share  (mrq)", 0)) if balance_sheet.get("Book Value Per Share  (mrq)") is not None else -1.0,
            # Cash Flow Statement
            float(cash_flow.get("Operating Cash Flow  (ttm)", 0)) if cash_flow.get("Operating Cash Flow  (ttm)") is not None else -1.0,
            float(cash_flow.get("Levered Free Cash Flow  (ttm)", 0)) if cash_flow.get("Levered Free Cash Flow  (ttm)") is not None else -1.0,
            # Stock Price History
            float(stock_history.get("Beta (5Y Monthly)", 0)) if stock_history.get("Beta (5Y Monthly)") is not None else -1.0,
            float(stock_history.get("52 Week Range3", 0)) if stock_history.get("52 Week Range3") is not None else -1.0,
            float(stock_history.get("S&P 500 52-Week Change3", 0)) if stock_history.get("S&P 500 52-Week Change3") is not None else -1.0,
            float(stock_history.get("52 Week High3", 0)) if stock_history.get("52 Week High3") is not None else -1.0,
            float(stock_history.get("52 Week Low3", 0)) if stock_history.get("52 Week Low3") is not None else -1.0,
            float(stock_history.get("50-Day Moving Average3", 0)) if stock_history.get("50-Day Moving Average3") is not None else -1.0,
            float(stock_history.get("200-Day Moving Average3", 0)) if stock_history.get("200-Day Moving Average3") is not None else -1.0,
            # Share Statistics
            float(share_stats.get("Avg Vol (3 month)3", 0)) if share_stats.get("Avg Vol (3 month)3") is not None else -1.0,
            float(share_stats.get("Avg Vol (10 day)3", 0)) if share_stats.get("Avg Vol (10 day)3") is not None else -1.0,
            float(share_stats.get("Shares Outstanding5", 0)) if share_stats.get("Shares Outstanding5") is not None else -1.0,
            float(share_stats.get("Implied Shares Outstanding6", 0)) if share_stats.get("Implied Shares Outstanding6") is not None else -1.0,
            float(share_stats.get("Float8", 0)) if share_stats.get("Float8") is not None else -1.0,
            float(share_stats.get("% Held by Insiders1", 0)) if share_stats.get("% Held by Insiders1") is not None else -1.0,
            float(share_stats.get("% Held by Institutions1", 0)) if share_stats.get("% Held by Institutions1") is not None else -1.0,
            float(share_stats.get("Shares Short (12/31/2024)4", 0)) if share_stats.get("Shares Short (12/31/2024)4") is not None else -1.0,
            float(share_stats.get("Short Ratio (12/31/2024)4", 0)) if share_stats.get("Short Ratio (12/31/2024)4") is not None else -1.0,
            float(share_stats.get("Short % of Float (12/31/2024)4", 0)) if share_stats.get("Short % of Float (12/31/2024)4") is not None else -1.0,
            float(share_stats.get("Short % of Shares Outstanding (12/31/2024)4", 0)) if share_stats.get("Short % of Shares Outstanding (12/31/2024)4") is not None else -1.0,
            float(share_stats.get("Shares Short (prior month 11/29/2024)4", 0)) if share_stats.get("Shares Short (prior month 11/29/2024)4") is not None else -1.0,
        ]
        for feature in result:
            if feature == -1:
                result.remove(feature)
        if len(result)>52:
            newData.append(company)
        results.append(len(result))
print(results[0:25])


In [ ]:
len(newData)

In [ ]:
nr = 0
for result in results:
    if result>53:
        nr+=1
nr

In [ ]:
import matplotlib.pyplot as plt
import statistics


mean = statistics.mean(results)
std_dev = statistics.stdev(results)

plt.figure(figsize=(10, 6))
plt.plot(results, 'bo-', label='Data Points')  # Corrected format string
plt.axhline(mean, color='r', linestyle='--', label=f'Mean ({mean:.2f})')
plt.axhline(mean + std_dev, color='g', linestyle=':', label=f'Mean +/- 1 SD ({std_dev:.2f})')
plt.axhline(mean - std_dev, color='g', linestyle=':')

plt.title('Data Points with Mean and Standard Deviation')
plt.xlabel('Index')
plt.ylabel('Value')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
import random
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline

data = [company for company in data if 'hist' in company and company['hist'] is not None]
random.shuffle(data)


test_size = int(len(data) * 0.20)
test_data = data[:test_size]
train_data = data[test_size:]


numeric_imputer = SimpleImputer(strategy='median')
robust_scaler = RobustScaler()

text_pipeline = Pipeline([
        ('tfidf', TfidfVectorizer(
            max_features=500,
            lowercase=True,
            ngram_range=(1, 2)
        )),
        ('svd', TruncatedSVD(n_components=100, random_state=42))
    ])

#Feature Engineering
def get_features(company):
    """Proper NaN handling for numeric features using 'or np.nan'."""
    import numpy as np
    features = []
    
    extra = company.get('extra', {})
    profitability = extra.get('Profitability', {})
    management = extra.get('Management Effectiveness', {})
    income_statement = extra.get("Income Statement", {})
    balance_sheet = extra.get("Balance Sheet", {})
    cash_flow = extra.get("Cash Flow Statement", {})
    stock_history = extra.get("Stock Price History", {})
    share_stats = extra.get("Share Statistics", {})
    
    # Direct features
    features.append(company.get('current_price', 0) or np.nan)
    features.append(company.get('Market Cap', 0) or np.nan)
    features.append(company.get('Enterprise Value', 0) or np.nan)
    features.append(company.get("Trailing P/E", 0) or np.nan)
    features.append(company.get("Forward P/E", 0) or np.nan)
    features.append(company.get("PEG Ratio (5yr expected)", 0) or np.nan)
    features.append(company.get("Price/Sales", 0) or np.nan)
    features.append(company.get("Price/Book", 0) or np.nan)
    features.append(company.get("Enterprise Value/Revenue", 0) or np.nan)
    features.append(company.get("Enterprise Value/EBITDA", 0) or np.nan)
    
    # Hist-based features (assumes getDataForPercentageRange is defined)
    val = getDataForPercentageRange(company.get("hist"), -100, -80)
    features.append(val[0] if val else np.nan)
    features.append(val[1] if val and len(val) > 1 else np.nan)
    val = getDataForPercentageRange(company.get("hist"), -60, -40)
    features.append(val[0] if val else np.nan)
    features.append(val[1] if val and len(val) > 1 else np.nan)
    val = getDataForPercentageRange(company.get("hist"), -40, -20)
    features.append(val[0] if val else np.nan)
    features.append(val[1] if val and len(val) > 1 else np.nan)
    val = getDataForPercentageRange(company.get("hist"), -20, 0)
    features.append(val[0] if val else np.nan)
    features.append(val[1] if val and len(val) > 1 else np.nan)
    
    # Profitability
    features.append(float(profitability.get("Profit Margin", 0) or np.nan))
    features.append(float(profitability.get("Operating Margin  (ttm)", 0) or np.nan))
    
    # Management
    features.append(float(management.get("Return on Assets  (ttm)", 0) or np.nan))
    features.append(float(management.get("Return on Equity  (ttm)", 0) or np.nan))
    
    # Income Statement
    features.append(float(income_statement.get("Revenue  (ttm)", 0) or np.nan))
    features.append(float(income_statement.get("Revenue Per Share  (ttm)", 0) or np.nan))
    features.append(float(income_statement.get("Quarterly Revenue Growth  (yoy)", 0) or np.nan))
    features.append(float(income_statement.get("Gross Profit  (ttm)", 0) or np.nan))
    features.append(float(income_statement.get("EBITDA", 0) or np.nan))
    features.append(float(income_statement.get("Net Income Avi to Common  (ttm)", 0) or np.nan))
    features.append(float(income_statement.get("Diluted EPS  (ttm)", 0) or np.nan))
    features.append(float(income_statement.get("Quarterly Earnings Growth  (yoy)", 0) or np.nan))
    
    # Balance Sheet
    features.append(float(balance_sheet.get("Total Cash  (mrq)", 0) or np.nan))
    features.append(float(balance_sheet.get("Total Cash Per Share  (mrq)", 0) or np.nan))
    features.append(float(balance_sheet.get("Total Debt  (mrq)", 0) or np.nan))
    features.append(float(balance_sheet.get("Total Debt/Equity  (mrq)", 0) or np.nan))
    features.append(float(balance_sheet.get("Current Ratio  (mrq)", 0) or np.nan))
    features.append(float(balance_sheet.get("Book Value Per Share  (mrq)", 0) or np.nan))
    
    # Cash Flow Statement
    features.append(float(cash_flow.get("Operating Cash Flow  (ttm)", 0) or np.nan))
    features.append(float(cash_flow.get("Levered Free Cash Flow  (ttm)", 0) or np.nan))
    
    # Stock Price History
    features.append(float(stock_history.get("Beta (5Y Monthly)", 0) or np.nan))
    features.append(float(stock_history.get("52 Week Range3", 0) or np.nan))
    features.append(float(stock_history.get("S&P 500 52-Week Change3", 0) or np.nan))
    features.append(float(stock_history.get("52 Week High3", 0) or np.nan))
    features.append(float(stock_history.get("52 Week Low3", 0) or np.nan))
    features.append(float(stock_history.get("50-Day Moving Average3", 0) or np.nan))
    features.append(float(stock_history.get("200-Day Moving Average3", 0) or np.nan))
    
    # Share Statistics
    features.append(float(share_stats.get("Avg Vol (3 month)3", 0) or np.nan))
    features.append(float(share_stats.get("Avg Vol (10 day)3", 0) or np.nan))
    features.append(float(share_stats.get("Shares Outstanding5", 0) or np.nan))
    features.append(float(share_stats.get("Implied Shares Outstanding6", 0) or np.nan))
    features.append(float(share_stats.get("Float8", 0) or np.nan))
    features.append(float(share_stats.get("% Held by Insiders1", 0) or np.nan))
    features.append(float(share_stats.get("% Held by Institutions1", 0) or np.nan))
    features.append(float(share_stats.get("Shares Short (12/31/2024)4", 0) or np.nan))
    features.append(float(share_stats.get("Short Ratio (12/31/2024)4", 0) or np.nan))
    features.append(float(share_stats.get("Short % of Float (12/31/2024)4", 0) or np.nan))
    features.append(float(share_stats.get("Short % of Shares Outstanding (12/31/2024)4", 0) or np.nan))
    features.append(float(share_stats.get("Shares Short (prior month 11/29/2024)4", 0) or np.nan))
    
    return features

def get_targets(company):
    """Handle target extraction with error checking"""
    try:
        hist = company['hist']
        ranges = [(0, 20), (20, 40), (40, 60), (60, 80), (80, 100), (10, 100)]
        lowhigh = []
        for r_start, r_end in ranges:
            data = getDataForPercentageRange(hist, r_start, r_end)
            lowhigh.extend(data[0:2])
        if len(lowhigh) != 12 or any(np.isnan(lowhigh)):
            return None
        return lowhigh
    except Exception as e:
        print(f"Error processing targets: {str(e)}")
        return None
import matplotlib.pyplot as plt
# Preprocessing data
def preprocess_data(data_subset, fit=False):
    X_numeric = []
    y = []
    descriptions = []
    
    for company in data_subset:
        features = get_features(company)
        X_numeric.append(features)
        
        targets = get_targets(company)
        if targets is None:
            continue
        y.append(targets)
        
        desc = str(company.get('company_description', '')).lower()
        descriptions.append(desc)
    
    # Numeric processing
    if fit:
        X_numeric = numeric_imputer.fit_transform(X_numeric)
        X_numeric = robust_scaler.fit_transform(X_numeric)
    else:
        X_numeric = numeric_imputer.transform(X_numeric)
        X_numeric = robust_scaler.transform(X_numeric)
    

    if fit:
        X_text = text_pipeline.fit_transform(descriptions)
    else:
        X_text = text_pipeline.transform(descriptions)
    
    # Combine features and handle remaining NaNs
    X = np.hstack([X_numeric, X_text])
    X = np.nan_to_num(X)
    
    return X, np.array(y)
